In [21]:
import numpy as np
import random
import matplotlib.pyplot as plt
import zfec
import scipy
import scipy.fft as fft


In [22]:
# globalne promenljive

fs = 44100
f0 = 10_000
df = 1_000
dt = .1
small_segment = 100

In [23]:
# Funkcije koje koristimo

class BitAppender:
    def __init__(self):
        self.buffer = bytearray()
        self.current_byte = 0
        self.bit_count = 0

    def append_bit(self, bit):
        if bit not in (0, 1):
            raise ValueError("Bit must be 0 or 1")

        # Add the bit to the current byte
        self.current_byte = (self.current_byte << 1) | bit
        self.bit_count += 1

        # If we have accumulated 8 bits, store the byte and reset
        if self.bit_count == 8:
            self.buffer.append(self.current_byte)
            self.current_byte = 0
            self.bit_count = 0

    def get_bytes(self):
        # If there are leftover bits, pad with zeros and add the final byte
        if self.bit_count > 0:
            self.current_byte <<= (8 - self.bit_count)
            self.buffer.append(self.current_byte)
        return bytes(self.buffer)

def GenerateRandom(length = 1000):
    bin_string = ""
    
    for _ in range(length):
        bin_string += '1' if bool(random.getrandbits(1)) else '0'
    
    return bin_string

def AddFEC(bin):

    # početni parametri za enkodiranje pomoću FECa
    k = len(bin) // 8 # broj paketa koji su potrebni za dekodiranje feca
    m = int(k * 1.3) # broj paketa koji su poslati
    bin_data = bytes(bin, 'utf-8')

    encoder = zfec.easyfec.Encoder(k,m)
    encoded = encoder.encode(bin_data)

    bin_string = ''.join(list(format(int(bin(int.from_bytes(byte_data, byteorder='big'))[2:]), "08") for byte_data in encoded))

    return bin_string

def GenerateSignal(bin_code : str):

    # učitavanje globalnih promenljiva
    global fs, f0, df, dt

    # pravljenje frekvencija za pakete
    f = [f0 - df // 2, f0 + df // 2]

    # priprema za pravljenje mfsk niza
    samples_per_symbol = int(int(fs) * dt)
    
    # pravljenje mfsk niza sa frekvencijama
    mfsk = []
    for bit in bin_code:
        mfsk += [f[int(bit)] for _ in range(samples_per_symbol)]
    mfsk = np.array(mfsk)
    
    # pravljenje cpmfsk signala
    delta_phi = mfsk * np.pi / (fs / 2.0)
    phi = np.cumsum(delta_phi)
    cpmfsk = np.sin(phi)

    return cpmfsk

def AddingNoise(sig, snr):

    # pravljenje šuma sa određenim SNRom
    sig_watts = np.mean(sig ** 2)
    sig_db = 10 * np.log10(sig_watts)
    noise_db = sig_db - snr
    noise_watts = 10 ** (noise_db / 10)
    noise = np.random.normal(0, np.sqrt(noise_watts), len(sig))
    
    # spajanje signala i šuma
    signal_with_noise = sig + noise

    return signal_with_noise

# demod, pure
def DemodulateSignal(signal, f0, f1, small_segment, sample_rate):
    bl = []
    for i in np.arange(0,len(signal)/sample_rate,dt):
        y1 = signal[int(i*sample_rate + small_segment):int((i+dt)*sample_rate) - small_segment]/1
        y1fft = scipy.fft.rfft(y1)

        n = int(dt * sample_rate)- 2 * small_segment
        freqs = fft.rfftfreq(n,1./sample_rate)

        maxf0 = 0
        maxf1 = 0

        for i in range(len(y1fft)):
            if  f0 - 300 < freqs[i] < f0 + 300 and y1fft[i] > maxf0:maxf0 = y1fft[i]
            if  f1 - 300 < freqs[i] < f1 + 300 and y1fft[i] > maxf1:maxf1 = y1fft[i]
            
        if maxf0 > maxf1: bl.append(0)
        if maxf0 < maxf1: bl.append(1)
    return bl

def RemoveFEC(bin):
    packs = []
    i = 0
    pack = BitAppender()
    while i < len(bin):
        pack.append_bit(int(bin[i]))
        i += 1
        if i == 8:
            packs.append()
            

# BER Kalkulacija, snr nalepljen
def BER(input_bin, output_bin, snr):
    # Ensure both lists are of the same length
    min_length = min(len(input_bin), len(output_bin))
    truncated_list1 = input_bin[:min_length]
    truncated_list2 = output_bin[:min_length]
    
    # Calculate the Bit Error Rate (BER)
    errors = sum(bit1 != bit2 for bit1, bit2 in zip(truncated_list1, truncated_list2))
    ber = errors / min_length
    
    # Attach the SNR value
    result = {"BER": ber, "SNR": snr}
    
    return result

In [26]:
BERs = []
snr_lower, snr_higher, snr_step = 1, -10, -.1
repeats_per_snr = 10
message_length = 8 * 1000

input_bin = GenerateRandom(message_length) # generisanje poruke
input_bin = AddFEC(input_bin)
for noise in np.arange(snr_lower, snr_higher, snr_step):
    for _ in range(repeats_per_snr):
        signal = AddingNoise(GenerateSignal(input_bin), noise) # modulacija signala
        
        output_bin = DemodulateSignal(signal, f0 - df // 2, f0 + df // 2, small_segment, fs) # demodulacija
        output_bin = RemoveFEC(output_bin)
        ber_snr = BER(list(map(int,input_bin)), output_bin, noise) # računanje BER/SNR
        BERs.append(ber_snr)
print(BERs)
# plotovanje
# fig, ax = plt.subplots(1, 1, figsize = (8, 2))
# ax.plot(BERs)

KeyboardInterrupt: 